In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

import pandas
import numpy
import matplotlib.pyplot as plt
import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.python.keras.models import Model
from keras.utils import plot_model

df = pandas.read_csv('largecategoricaltestdata.csv', sep=';')
    
categoricalConversions = {'yes': 1, 'no': 0}
df['y'] = df['y'].map(categoricalConversions)
y = df['y']

df = pandas.get_dummies(df,df.columns[df.dtypes == 'object'])
X = df.drop('y', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

input_layer = Input(shape=(X.shape[1],)) ;'63'
dl1 = Dense(32, activation='relu')(input_layer)
dl2 = Dense(16, activation='relu')(dl1)
output_layer = Dense(1, activation='sigmoid')(dl2)

model = Model(inputs = input_layer, outputs=output_layer)
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

History = model.fit(X_train, y_train, epochs=75, validation_split=0.2)
loss, accuracy = model.evaluate(X_test,  y_test) 
print('\n'+'Accuracy:', accuracy)
print('Loss:', loss)

In [ ]:
predicted = model.predict(X_test)

result_noyes = []
result_01 = []

for index in range(len(predicted)):
    current = predicted[index][0]
    if (current < 0.5): 
        result_noyes.append('No') 
        result_01.append(0)
    elif (current > 0.5): 
        result_noyes.append('Yes')
        result_01.append(1) 
    else: 
        result_noyes.append('Neutral')
        result_01.append(0)

'print(History.history.keys())'

'''
# summarize history for accuracy
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('accuracy vs epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('loss vs epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# combined
pandas.DataFrame(History.history).plot(figsize=(7,5))
plt.show()
'''

print(metrics.classification_report(y_test, result_01))

confusion_matrix = metrics.confusion_matrix(y_test, result_01)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=[False,True])

cm_display.plot()
plt.show()

Accuracy = metrics.accuracy_score(y_test, result_01)
Precision = metrics.precision_score(y_test, result_01, pos_label=0)
Specificity = metrics.recall_score(y_test, result_01, pos_label=0)
F1_score = metrics.f1_score(y_test, result_01, pos_label=0)

#metrics:
print({"Accuracy":Accuracy,"Precision":Precision,"Specificity":Specificity,"F1_score":F1_score})
